In [7]:
import pandas as pd
import numpy as np

# ==============================
# User Settings
# ==============================
CSV_FILE   = r"C:\Users\krish\Desktop\SpatialCARE\Hourly\pasig_hourly_corrected.csv"
OUT_FILE   = r"C:\Users\krish\Desktop\SpatialCARE\Hourly\pasig_hourly_aqi.csv"

# ==============================
# Load dataset
# ==============================
df = pd.read_csv(CSV_FILE)

# Combine Date + Time → datetime
if "Date" in df.columns and "Time" in df.columns:
    df["datetime"] = pd.to_datetime(df["Date"] + " " + df["Time"])
else:
    raise ValueError("❌ Expected 'Date' and 'Time' columns in CSV.")

# ==============================
# PM2.5 → AQI conversion
# ==============================
def pm25_to_aqi(pm25):
    breakpoints = [
        (0.0, 12.0, 0, 50),
        (12.1, 35.4, 51, 100),
        (35.5, 55.4, 101, 150),
        (55.5, 150.4, 151, 200),
        (150.5, 250.4, 201, 300),
        (250.5, 500.0, 301, 500),
    ]
    for (clow, chigh, alow, ahigh) in breakpoints:
        if clow <= pm25 <= chigh:
            return round((ahigh - alow) / (chigh - clow) * (pm25 - clow) + alow, 1)
    return np.nan

# Apply conversion
df["AQI"] = df["pm25"].apply(pm25_to_aqi)

# ==============================
# Save new dataset
# ==============================
df.to_csv(OUT_FILE, index=False)
print(f"✅ AQI values added and saved to: {OUT_FILE}")
print(df.head())

✅ AQI values added and saved to: C:\Users\krish\Desktop\SpatialCARE\Hourly\pasig_hourly_aqi.csv
         Date     Time                  location_name  latitude  longitude  \
0  2025-01-01  0:00:00  Country Lodge, Pasig [EMBNCR]     14.57     121.07   
1  2025-01-01  1:00:00  Country Lodge, Pasig [EMBNCR]     14.57     121.07   
2  2025-01-01  2:00:00  Country Lodge, Pasig [EMBNCR]     14.57     121.07   
3  2025-01-01  3:00:00  Country Lodge, Pasig [EMBNCR]     14.57     121.07   
4  2025-01-01  4:00:00  Country Lodge, Pasig [EMBNCR]     14.57     121.07   

     pm25            datetime    AQI  
0  273.04 2025-01-01 00:00:00  319.0  
1  314.04 2025-01-01 01:00:00  351.7  
2  138.58 2025-01-01 02:00:00  193.9  
3  103.87 2025-01-01 03:00:00  176.0  
4  108.87 2025-01-01 04:00:00  178.6  
